In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy
import random
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/Shared\ drives/ISE540/Data/Problem2

/content/drive/Shared drives/ISE540/Data/Problem2


In [ ]:
df_unbalanced = pd.read_csv("Sampled_unbalanced.csv")

In [ ]:
df_unbalanced

,Unnamed: 0,ID,Text,Domain,URL,Content,Match
0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...,1
1,1,1.324130e+18,"In any season, getting sick is tough. But rig...",Medical,https://t.co/OsHSfvef52,AboutPressCopyrightContact usCreatorsAdvertise...,1
2,2,1.324130e+18,@FatEmperor And the lastest vicim of the swine...,Medical,https://t.co/HPkErToAHc,"To date, more than 80 cases have been lodged b...",1
3,3,1.324130e+18,Fargo Cass Public Health holds additional flu ...,Medical,https://t.co/ztEuaglz07,FARGO — Due to the positive response to last w...,1
4,4,1.324130e+18,Kitchener man Hosherdar Polad says he feels di...,Medical,https://t.co/XGEhe89mhn,WATERLOO — Getting a flu shot seemed especiall...,1
...,...,...,...,...,...,...,...
13175,13175,1.324440e+18,TONIGHT in Milwaukee @thegaywhostrayd @milknco...,unknown,https://t.co/2jyuPOHk1T,Kingdom Two Crowns ($9.99) from Raw Fury is so...,0
13176,13176,1.324440e+18,TONIGHT in Milwaukee @thegaywhostrayd @milknco...,unknown,https://t.co/7sBC8EXv9Y,Server IP - Copy to Minecraft Client to Play: ...,0
13177,13177,1.324440e+18,TONIGHT in Milwaukee @thegaywhostrayd @milknco...,unknown,https://t.co/bi2bhLBCGI,Hallo wir sind eine Gruppe von gewürfelten gam...,0
13178,13178,1.324440e+18,TONIGHT in Milwaukee @thegaywhostrayd @milknco...,unknown,https://t.co/rJE5J81PAu,|Score confirmed by Crtalleyfb at 9:36:47 PM||...,0


In [ ]:
df_balanced = pd.read_csv("Sampled_balanced.csv")

In [ ]:
df_balanced

,Unnamed: 0,index,ID,Text,Domain,URL,Content,Match
0,0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...,1
1,1,1,1.324130e+18,"In any season, getting sick is tough. But rig...",Medical,https://t.co/OsHSfvef52,AboutPressCopyrightContact usCreatorsAdvertise...,1
2,2,2,1.324130e+18,@FatEmperor And the lastest vicim of the swine...,Medical,https://t.co/HPkErToAHc,"To date, more than 80 cases have been lodged b...",1
3,3,3,1.324130e+18,Fargo Cass Public Health holds additional flu ...,Medical,https://t.co/ztEuaglz07,FARGO — Due to the positive response to last w...,1
4,4,4,1.324130e+18,Kitchener man Hosherdar Polad says he feels di...,Medical,https://t.co/XGEhe89mhn,WATERLOO — Getting a flu shot seemed especiall...,1
...,...,...,...,...,...,...,...,...
23719,23719,1313,1.324440e+18,This Type of Meat Is Linked to Increased Cance...,unknown,https://t.co/2u2KRJSDj9,This Type of Meat Is Linked to Increased Cance...,1
23720,23720,1314,1.324440e+18,@BluebirdsofMO @collhistgarden @HawleyMO @GOP ...,unknown,https://t.co/4vbi3brgeq,Missouri Attorney General Josh Hawley doubled ...,1
23721,23721,1315,1.324440e+18,Quite a mystery!,unknown,https://t.co/vycImaqSZ3,OU Medicine expert looks at why Oklahoma's see...,1
23722,23722,1316,1.324440e+18,I wrote this back in March along with some oth...,unknown,https://t.co/UgrY57yXvP,At the moment I think it’s fair to say that ma...,1


In [ ]:
vectorizer_tweet = TfidfVectorizer(max_df = 200,max_features=300)
vectorizer_content = TfidfVectorizer(max_df = 200,max_features=300)
tweet_tfidf = vectorizer_tweet.fit_transform(df_unbalanced["Text"])
content_tfidf = vectorizer_content.fit_transform(df_unbalanced["Content"])

In [ ]:
import scipy.sparse as sp

In [ ]:
A_tweet_tfidf = tweet_tfidf.A
A_content_tfidf = content_tfidf.A

In [ ]:
concat_tfidf =np.hstack((A_tweet_tfidf, A_content_tfidf))

In [ ]:
del A_content_tfidf
del A_tweet_tfidf

In [ ]:
y = df_unbalanced["Match"]

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(concat_tfidf, y, test_size=0.3, random_state=42,stratify = y)

In [ ]:
X_train.shape

(9226, 600)

Logistic Regreesion

In [ ]:
clf1 = LogisticRegression(class_weight='balanced').fit(X_train, y_train)

In [ ]:
y_pred1 =clf1.predict(X_test)
clf1.score(X_test,y_test)

0.5010116337885685

In [ ]:
f1_score(y_test, y_pred1)

0.06092336982389339

In [ ]:
for i in [10,20,30,50,100]:
    
    model1 = LogisticRegression(class_weight={1: i,0:1},max_iter=10000)
    model1.fit(X_train, y_train)
    
    y_pred=model1.predict(X_test)
    print("model accuracy is: ",model1.score(X_test,y_test))
    print(f1_score(y_test, y_pred))

    #print('recall',recall_score(y_test, y_pred))

model accuracy is:  0.38391502276176026
0.08900523560209422
model accuracy is:  0.16009104704097116
0.14340985297910755
model accuracy is:  0.1294891249367729
0.15761135584924132
model accuracy is:  0.11633788568538189
0.16571155682903532
model accuracy is:  0.10900354071825999
0.17203290246768507


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(class_weight='balanced')
model2.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred=model2.predict(X_test)
model2.score(X_test,y_test)

0.6724835609509358

In [ ]:
f1_score(y_test, y_pred)

0.10006949270326616

In [ ]:
for i in [10,20,30,50,100]:
    
    model2 = DecisionTreeClassifier(class_weight={1: i,0:1})
    model2.fit(X_train, y_train)
    
    y_pred=model2.predict(X_test)
    print("accuracy is",model2.score(X_test,y_test))
    print(f1_score(y_test, y_pred))

accuracy is 0.6707132018209409
0.10330578512396695
accuracy is 0.6679312089023773
0.10497614178595775
accuracy is 0.6646433990895296
0.11126005361930294
accuracy is 0.6679312089023773
0.10006854009595613
accuracy is 0.6722306525037937
0.10620689655172415


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
param_grid = [{'max_depth':[2,4,8,10]}]
kfold = StratifiedKFold(n_splits=10, shuffle = True)
grid = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=4, cv = kfold, scoring = 'accuracy')

In [ ]:
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                         

In [ ]:
grid_values = list(grid.best_params_.values())

In [ ]:
grid_values

[2]

In [ ]:
model3 = RandomForestClassifier(n_estimators = 500, max_depth=2, class_weight='balanced')
model3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred=model3.predict(X_test)
model3.score(X_test,y_test)

0.6302478502781993

In [ ]:
f1_score(y_test, y_pred)

0.07818411097099622

Re-sampling the train set

In [ ]:
y_train.value_counts()[1]

923

In [ ]:
X_train_rs = X_train
y_train_rs = y_train

In [ ]:
diff = y_train.value_counts()[0] - y_train.value_counts()[1]
diff

7380

In [ ]:
ones = X_train_rs[y_train_rs == 1]

In [ ]:
for i in range(0,diff):
  temp = random.choice(ones)
  X_train_rs = np.vstack([X_train_rs, temp])
  y_train_rs = np.append(y_train_rs,1)

In [ ]:
X_train_rs.shape

(16606, 600)

In [ ]:
np.bincount(y_train_rs)

array([8303, 8303])

In [ ]:
randomize = np.arange(len(X_train_rs))
np.random.shuffle(randomize)
X_train_rs_r = X_train_rs[randomize]
y_train_rs_r = y_train_rs[randomize]

In [ ]:
X_train_rs_r.shape

(16606, 600)

Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
model4 = BernoulliNB()
model4.fit(X_train_rs_r, y_train_rs_r)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
y_pred = model4.predict(X_test)
model4.score(X_test,y_test)

0.5735963581183612

In [ ]:
f1_score(y_test, y_pred)

0.07056229327453142

MLP model


In [ ]:
from sklearn.neural_network import MLPClassifier
model5 = MLPClassifier()
model5.fit(X_train_rs_r, y_train_rs_r)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred = model5.predict(X_test)
model5.score(X_test,y_test)

0.7503793626707133

In [ ]:
f1_score(y_test, y_pred)

0.13799126637554585

In [ ]:
ppt_str = df_unbalanced["Text"][0]

In [ ]:
df_unbalanced[df_unbalanced["Text"] == ppt_str]

,Unnamed: 0,ID,Text,Domain,URL,Content,Match
0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...,1
1318,1318,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/Ci710zibuf,"Free flu shots: Memphis, Germantown to have dr...",0
1319,1319,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/O8kGQgNVhd,Federal regulators have cleared dozens of AI p...,0
1320,1320,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/DTNpYVF3cs,It was feared by many to be the perfect winter...,0
1321,1321,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/9fZb8g5t2h,The U.S. Food and Drug Administration (FDA) is...,0
1322,1322,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/fRszLS1gwv,Follow onAdvertisement(PETER PARKS/AFP/Getty I...,0
1323,1323,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/NGX4qeVz4b,Flu activity lower than normal: 6 notes from C...,0
1324,1324,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/9vDHReOzJx,The High Court has approved a settlement in th...,0
1325,1325,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/wELAhDbP9S,"The Costa Smeralda, the beach resort on the no...",0
1326,1326,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vw9iMYTBW2,- PMID: 31607599- PMCID: PMC7126676- DOI: 10.1...,0


Random Guess

In [ ]:
import random

In [ ]:
y_test.shape

(3954,)

In [ ]:
random_y = []
for i in range(0,len(y_test)):
  random_y.append(random.randint(0,1))

In [ ]:
f1_score(y_test, random_y)

0.15917843388960204

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,random_y)

0.5030349013657056